## User guide
https://pcse.readthedocs.io/en/stable/user_guide.html#getting-started


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pp
from datetime import datetime

## Running PCSE/WOFOST with custom input data
For running PCSE/WOFOST (and PCSE models in general) with your own data sources you need three different types of inputs:

1. Model parameters that parameterize the different model components. These parameters usually consist of :
   - a set of crop parameters (or multiple sets in case of crop rotations), 
   - a set of soil parameters and 
   - a set of site parameters: ancillary parameters that are specific for a location.

2. Driving variables represented by weather data which can be derived from various sources.
3. Agromanagement actions which specify the farm activities that will take place on the field that is simulated by PCSE.

In [ ]:
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider
from pathlib import Path

from nevergrad.functions.irrigation.irrigation import get_soil_data

## 1. Model parameters
### 1.1. Crop parameters

In [ ]:
cropfile = "https://raw.githubusercontent.com/ajwdewit/WOFOST_crop_parameters/master/"
cropdata = YAMLCropDataProvider(repository=cropfile)

In [ ]:
pp(cropdata.get_crops_varieties())

### 1.2. Soil parameters
- soil type 
- soil physical properties

In [ ]:
# we will use the water balance for 
# freely draining soils and use the soil file for medium fine sand
soildata = get_soil_data()

In [ ]:
pp(soildata)

### 1.3. Site parameters

In [ ]:
from pcse.util import WOFOST71SiteDataProvider

In [ ]:
# the initial conditions of the water balance such as the initial soil moisture content
sitedata = WOFOST71SiteDataProvider(
    WAV=100, # initial soil moisture content
    CO2=360, # the atmospheric CO2 concentration
)

In [ ]:
expected_sitedata = {
    'IFUNRN': 0,
    'NOTINF': 0,
    'SSI': 0.0, # the initial surface storage
    'SSMAX': 0.0, # the maximum surface storage
    'WAV': 100.0,
    'SMLIM': 0.4
}
assert sitedata == expected_sitedata

### 1.4. Combine them

In [ ]:
from pcse.base import ParameterProvider

In [ ]:
parameters = ParameterProvider(cropdata=cropdata, soildata=soildata, sitedata=sitedata)

## 2. Weather data

In [ ]:
from pcse.db import NASAPowerWeatherDataProvider

In [ ]:
wdp = NASAPowerWeatherDataProvider(
    latitude=51.97, longitude=5.67,
)

In [ ]:
print(wdp)

In [ ]:
crop_start_date = datetime.strptime("2006-03-31", "%Y-%m-%d").date()
crop_end_date = datetime.strptime("2006-10-20", "%Y-%m-%d").date()

In [ ]:
df_weatherdataprovider = pd.DataFrame(wdp.export()).set_index("DAY")

mask = (df_weatherdataprovider.index >= crop_start_date) & (df_weatherdataprovider.index <= crop_end_date)
df_weatherdataprovider.loc[mask, ["RAIN"]].plot()

## 3. Agromanagement

https://github.com/ajwdewit/pcse_notebooks/blob/master/06_advanced_agromanagement_with_PCSE.ipynb

In [ ]:
import yaml

In [ ]:
yaml_agro_template = """
- 2006-01-01:
    CropCalendar:
        crop_name: sugarbeet
        variety_name: Sugarbeet_603
        crop_start_date: 2006-03-31
        crop_start_type: emergence
        crop_end_date: 2006-10-20
        crop_end_type: harvest
        max_duration: 300
    TimedEvents: null
    StateEvents: null
"""

agromanagement = yaml.safe_load(yaml_agro_template)

In [ ]:
yaml_agro_irrigation_template = """
- 2006-01-01:
    CropCalendar:
        crop_name: sugarbeet
        variety_name: Sugarbeet_603
        crop_start_date: 2006-03-31
        crop_start_type: emergence
        crop_end_date: 2006-10-20
        crop_end_type: harvest
        max_duration: 300
    TimedEvents:
        -   event_signal: irrigate
            name: Irrigation application table
            comment: All irrigation amounts in cm
            events_table:
            - 2006-07-01: {amount: 10, efficiency: 0.7}
            - 2006-08-01: {amount: 10, efficiency: 0.7}
    StateEvents: null
"""
agromanagement_irrigation = yaml.safe_load(yaml_agro_irrigation_template)

In [ ]:
def get_irrigation_events(campaign):
    event_cfgs = []

    for campaign_start_date, schedule in campaign.items():
        events_cfg = schedule['TimedEvents']
        irrigation_events = events_cfg[0]["events_table"]
        event_cfgs.extend(
            parse_irrigation_events(irrigation_events)
        )
    return event_cfgs

def parse_irrigation_events(events):
    event_cfgs = []
    
    for event in events:
        for date, irrigation_water_in_cm in event.items():
            event_cfgs.append((date, irrigation_water_in_cm))
    return event_cfgs

In [ ]:
first_campaign = agromanagement_irrigation[0]
irrigation_events = get_irrigation_events(first_campaign)
irrigation_events

## 4. Initializing WOFOST model

In [ ]:
from pcse.models import Wofost72_WLP_FD, Wofost72_PP

In [ ]:
wofost_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement)
wofost_irrigation_sim = Wofost72_WLP_FD(parameters, wdp, agromanagement_irrigation)

In [ ]:
wofost_sim.run_till_terminate()
wofost_irrigation_sim.run_till_terminate()

In [ ]:
outputs = wofost_sim.get_output()
df_output = pd.DataFrame(outputs)

outputs_irrigation = wofost_irrigation_sim.get_output()
df_output_irrigation = pd.DataFrame(outputs_irrigation)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "LAI"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")
for event_cfg in irrigation_events:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']} cm")

# df_weatherdataprovider.loc[mask, ["RAIN"]].plot(ax=ax)
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15, 5))
var = "SM"
df_output.set_index("day")[var].plot(ax=ax)
df_output_irrigation.set_index("day")[var].plot(ax=ax, label=f"{var} irrigation")

for event_cfg in irrigation_events:
    event_date, irrigation_water_in_cm = event_cfg
    ax.axvline(x=event_date, c="red", label=f"irrigation event: {irrigation_water_in_cm['amount']} cm")

# df_weatherdataprovider.loc[mask, ["RAIN"]].plot(ax=ax)
plt.legend()
plt.show()

## Optimize a simple irrigation planning

In [ ]:
import scipy
import numpy as np

In [ ]:
agro_yaml = """
- 2006-01-01:
    CropCalendar:
        crop_name: sugarbeet
        variety_name: Sugarbeet_603
        crop_start_date: 2006-03-31
        crop_start_type: emergence
        crop_end_date: 2006-10-20
        crop_end_type: harvest
        max_duration: 300
    TimedEvents:
    -   event_signal: irrigate
        name: Irrigation application table
        comment: All irrigation amounts in cm
        events_table:
        - 2006-07-01: {{amount: {a0}, efficiency: 0.7}}
        - 2006-08-01: {{amount: 10, efficiency: 0.7}}
    StateEvents: null
"""

def objective_function(
    irrigation_volume_in_cm: float,
    parameterprovider=parameters,
    weatherdataprovider=wdp,
) -> float:
    if isinstance(irrigation_volume_in_cm, np.ndarray):
        assert len(irrigation_volume_in_cm) == 1
        irrigation_volume_in_cm = irrigation_volume_in_cm[0]
    agromanagement = yaml.safe_load(
        agro_yaml.format(a0=irrigation_volume_in_cm)
    )
    wofost = Wofost72_WLP_FD(parameterprovider, weatherdataprovider, agromanagement)
    wofost.run_till_terminate()
    output = wofost.get_output()
    df_output = pd.DataFrame(output)
    return -df_output["LAI"].sum()

In [ ]:
irrigation_volumes = np.arange(20)
loss = [objective_function(irrigation_volume) for irrigation_volume in irrigation_volumes]

In [ ]:
plt.plot(irrigation_volumes, loss)

In [ ]:
scipy.optimize.minimize(
    fun=objective_function,
    x0=5
)